##### 1단계: LangChain 설치 및 설정

gcloud cli 인증


In [1]:
#gcloud auth application-default login

##### 2단계: Groq Qwen2.5 32B 설치 및 설정 지침  -> 대신 GCP gemini로 변경

In [2]:
import os
from langchain_google_vertexai import ChatVertexAI


# Vertex AI Gemini 기반 LLM
llm = ChatVertexAI(
    model="gemini-2.5-flash",   # 또는 "gemini-2.5-pro"
    temperature=0.0,
    
)


##### 3단계: NVIDIA bge-m3 설치 및 설정 -> 대신 vertex 임베딩

In [3]:
#pip install -qU langchain-nvidia-ai-endpoints


In [4]:

from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model_name="text-embedding-004")

##### 4단계: Milvus 설치 및 설정 -> 대신 faiss

##### 5단계: RAG 챗봇 구축

In [11]:
import os
import json
import faiss

import vertexai
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_core.prompts import ChatPromptTemplate



FAISS_INDEX_PATH = "paper.faiss"
DATA_JSON_PATH   = "paper.json"
MAP_JSON_PATH    = "paper_to_model.json"

vector_store = None  

try:
    # 1. FAISS 인덱스 로드
    print(f"[INFO] FAISS 인덱스 '{FAISS_INDEX_PATH}' 로드 중...")
    index = faiss.read_index(FAISS_INDEX_PATH)

    # 2. JSON 데이터 로드
    print(f"[INFO] 원본 데이터 '{DATA_JSON_PATH}' 및 맵 '{MAP_JSON_PATH}' 로드 중...")
    with open(DATA_JSON_PATH, "r", encoding="utf-8") as f:
        paper_DATA = json.load(f)

    with open(MAP_JSON_PATH, "r", encoding="utf-8") as f:
        # 예: { "0": "BERT", "1": "GPT-3", ... }
        index_to_model_map = json.load(f)

    # 3. LangChain Docstore 재구성
    print("[INFO] LangChain Docstore 재구성 중...")
    docstore_dict = {}
    index_to_docstore_id = {}

    for str_idx, model_name in index_to_model_map.items():
        int_idx = int(str_idx)

        # docstore ID (여기서는 모델명을 그대로 사용)
        doc_id = model_name

        info = paper_DATA[model_name]

        # build_database.py와 동일한 형태라고 가정
        text = f"모델명: {model_name}. " + " ".join(
            f"{key}: {value}" for key, value in info.items() if value is not None
        )

        metadata = {"model_name": model_name, **info}
        doc = Document(page_content=text, metadata=metadata)

        docstore_dict[doc_id] = doc
        index_to_docstore_id[int_idx] = doc_id

    docstore = InMemoryDocstore(docstore_dict)

    # 4. LangChain FAISS VectorStore 생성
    print("[INFO] LangChain FAISS VectorStore 생성 중...")
    vector_store = FAISS(
        embedding_function=embeddings,
        index=index,
        docstore=docstore,
        index_to_docstore_id=index_to_docstore_id,
    )

    print("[INFO] FAISS 인덱스 수동 로드 및 재구성 완료")

except FileNotFoundError as e:
    print(f"[오류] 필수 파일을 찾을 수 없습니다: {e}")
    print("paper.faiss, paper.json, paper_to_model.json 파일 경로를 확인하세요.")
except Exception as e:
    import traceback
    print(f"[오류] FAISS 로드 중 예외 발생: {e}")
    traceback.print_exc()


[INFO] FAISS 인덱스 'paper.faiss' 로드 중...
[INFO] 원본 데이터 'paper.json' 및 맵 'paper_to_model.json' 로드 중...
[INFO] LangChain Docstore 재구성 중...
[INFO] LangChain FAISS VectorStore 생성 중...
[INFO] FAISS 인덱스 수동 로드 및 재구성 완료


프롬프트 정의

In [12]:


print("[INFO] RAG 프롬프트 정의 중...")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 논문/모델 정보를 설명하는 어시스턴트입니다. "
            "주어진 context(논문 메타데이터와 설명)에 근거해서만 답변하세요. "
            "모르면 모른다고 말하세요."
        ),
        (
            "human",
            "질문: {question}\n\n"
            "다음은 검색된 논문/모델 정보입니다:\n"
            "{context}\n\n"
            "이 정보를 바탕으로 한국어로 자세히 답변해 주세요."
        ),
    ]
)

print("[INFO] 프롬프트 설정 완료")


[INFO] RAG 프롬프트 정의 중...
[INFO] 프롬프트 설정 완료


(2) RAG 한 번 호출하는 함수만 두기

In [13]:
# ===============================
# RAG 함수 정의
# ===============================

def rag_answer(question: str, k: int = 4):
    """FAISS에서 관련 문서를 검색하고, LLM으로 답변을 생성하는 단일 RAG 함수."""
    if vector_store is None:
        raise RuntimeError("vector_store가 초기화되지 않았습니다. FAISS 로드 셀을 먼저 확인하세요.")

    print(f"[RAG] 검색 질의: {question}")
    docs = vector_store.similarity_search(question, k=k)
    print(f"[RAG] 검색된 문서 수: {len(docs)}")

    # 검색된 문서 내용을 하나의 문자열로 합치기
    context_text = "\n\n".join(doc.page_content for doc in docs)

    # 프롬프트 + LLM 호출
    messages = prompt.invoke({"question": question, "context": context_text})
    response = llm.invoke(messages)

    return response.content, docs


In [14]:

print("\n--- 논문 챗봇 (RAG: Simple Version) ---")
test_question = "BERT 논문이 뭐야?"

if vector_store is None:
    print("[오류] 벡터 스토어 로딩에 실패하여 RAG를 실행할 수 없습니다. 위 셀의 에러 메시지를 먼저 확인하세요.")
else:
    try:
        print(f"\n[You]: {test_question}")
        answer, used_docs = rag_answer(test_question, k=4)

        print(f"\n[Bot]: {answer}")

        print("\n--- 참고한 문서 ---")
        for i, doc in enumerate(used_docs):
            model_name = doc.metadata.get("model_name", "Unknown")
            preview = doc.page_content[:120].replace("\n", " ")
            print(f"[{i+1}] {model_name}: {preview}...")

    except Exception as e:
        import traceback
        print(f"[오류] RAG 실행 중 예외 발생: {e}")
        traceback.print_exc()



--- 논문 챗봇 (RAG: Simple Version) ---

[You]: BERT 논문이 뭐야?
[RAG] 검색 질의: BERT 논문이 뭐야?
[RAG] 검색된 문서 수: 3

[Bot]: 네, 제공된 정보에 따르면 BERT 논문은 다음과 같습니다:

*   **모델명:** BERT
*   **제목:** BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
*   **저자:** Jacob Devlin et al.
*   **발표 연도:** 2018년
*   **주요 태스크:** 언어 이해 (Language Understanding)
*   **핵심 키워드:** Transformer, Pre-training, Bidirectional (트랜스포머, 사전 훈련, 양방향)

**요약:** BERT는 양방향 트랜스포머(Bidirectional Transformers)를 사전 훈련(Pre-training)하여 다양한 자연어 처리 태스크에서 최고 성능(SOTA)을 달성한 모델입니다.

--- 참고한 문서 ---
[1] BERT: 모델명: BERT. title: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding author: Jacob Devlin ...
[2] GPT-3: 모델명: GPT-3. title: Language Models are Few-Shot Learners author: Tom B. Brown et al. year: 2020 task: Language Generatio...
[3] ResNet: 모델명: ResNet. title: Deep Residual Learning for Image Recognition author: Kaiming He et al. year: 2015 task: Image Recogn...


In [15]:

print("\n--- 논문 챗봇 (RAG: Simple Version) ---")
test_question = "BERT 논문 작가가 누구야?"

if vector_store is None:
    print("[오류] 벡터 스토어 로딩에 실패하여 RAG를 실행할 수 없습니다. 위 셀의 에러 메시지를 먼저 확인하세요.")
else:
    try:
        print(f"\n[You]: {test_question}")
        answer, used_docs = rag_answer(test_question, k=4)

        print(f"\n[Bot]: {answer}")

        print("\n--- 참고한 문서 ---")
        for i, doc in enumerate(used_docs):
            model_name = doc.metadata.get("model_name", "Unknown")
            preview = doc.page_content[:120].replace("\n", " ")
            print(f"[{i+1}] {model_name}: {preview}...")

    except Exception as e:
        import traceback
        print(f"[오류] RAG 실행 중 예외 발생: {e}")
        traceback.print_exc()



--- 논문 챗봇 (RAG: Simple Version) ---

[You]: BERT 논문 작가가 누구야?
[RAG] 검색 질의: BERT 논문 작가가 누구야?
[RAG] 검색된 문서 수: 3

[Bot]: BERT 논문의 작가는 **Jacob Devlin 외 다수**입니다.

--- 참고한 문서 ---
[1] BERT: 모델명: BERT. title: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding author: Jacob Devlin ...
[2] GPT-3: 모델명: GPT-3. title: Language Models are Few-Shot Learners author: Tom B. Brown et al. year: 2020 task: Language Generatio...
[3] ResNet: 모델명: ResNet. title: Deep Residual Learning for Image Recognition author: Kaiming He et al. year: 2015 task: Image Recogn...
